# Introduction

Logging tool is a useful tool to visualize the system dynamics. Drake's VectorLog tool helps on this Topic.

Fisrtly, let's create a diagram that contains some dynamics.

$y = \int{2}dt$

In [ ]:
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.primitives import Integrator, Gain

builder = DiagramBuilder()
# integrator with size=1 (one element to be integrated)
integrator = Integrator(1)
gain = Gain(k=2, size=1)
builder.AddSystem(integrator)
builder.AddSystem(gain)
builder.Connect(gain.GetOutputPort("y0"), integrator.GetInputPort("u0"))
builder.ExportInput(gain.GetInputPort("u0"))
builder.ExportOutput(integrator.GetOutputPort("y0"))

Before building the diagram, we need to add logger to log the desired signal (output). To use LogVectorOutput, we need to give the builder it belongs to. So we need to instantiate the logger befor calling the Build() method

In [ ]:
from pydrake.systems.primitives import LogVectorOutput

logger = LogVectorOutput(integrator.GetOutputPort("y0"), builder)
logger.set_name("logger")

Now build the whole diagram (with the logger)

In [ ]:
diagram = builder.Build()

Create a simulator and run the simulation for 1 second

In [ ]:
from pydrake.systems.analysis import Simulator

simulator = Simulator(diagram)
context = simulator.get_mutable_context()
diagram.get_input_port(0).FixValue(context, 1)
simulator.AdvanceTo(1.0)

The logged data are stored in the simulator context, after the simulation finishes, we can extract it. 

In [ ]:
log = logger.FindLog(simulator.get_context())
t = log.sample_times()

To visualize the result, we use matplotlib

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
# Plot theta.
plt.plot(t, log.data()[0,:],'.-')
plt.show()